### <span style="color:#800000">NER Tagging</span>

### <span style="color:#FF00FF">Import libraries</span>

In [1]:
import pandas as pd
import re

from nltk.tokenize import word_tokenize 
from sklearn.feature_extraction.text import CountVectorizer

import spacy
from spacy.training import offsets_to_biluo_tags

pd.set_option('display.max_colwidth', 500)

### <span style="color:#FF00FF">Loading Datasets</span>

In [2]:
# Reading data and creating the dataframe 
job = pd.read_csv("../data/jobdescriptions_clean.csv")

# Checking the first 5 rows of the data
job.head()

,domain,link,title,company,experience,salary,location,description,role,industry_type,qualification,functional_area,employment_type,role_category,skills,clean_description
0,information-technology-jobs,https://www.naukri.com/job-listings-head-information-technology-bright-consultant-delhi-ncr-15-to-25-years-150321905374?src=jobsearchDesk&sid=16164857916548527&xp=1&px=1,Head - Information Technology,Bright Consultant,15 - 25 years,Not Disclosed,Delhi / NCR,": - manage all it verticals of the company like it applications, infrastructure and security. - leverage it shared service organization for deployment of new it solutions - develop organization level it strategies in an as-is & to-be scenario and align to the business objectives in a cost-efficient manner - analyse existing it infrastructure and areas of improvements around hardware, operating system, communications, software applications, data processing, security system and implementing t...","Head/VP/GM-Technology(IT)/CTO,","Industrial Products, Heavy Machinery,","['UG :Any Graduate in Any Specialization', 'PG :Any Postgraduate in Any Specialization', 'Doctorate :Doctorate Not Required']","IT Software - Network Administration, Security,","Full Time, Permanent",Senior Management,"['Network Security', 'Information Security', 'Chief Information Security Officer', 'IT Security']",manage verticals company like applications infrastructure security leverage shared service organization deployment new solutions develop organization level strategies scenario align business objectives cost efficient manner analyse existing infrastructure areas improvements around hardware operating system communications software applications data processing security system implementing develop implement overall security ensure utmost protection privacy organization information data identify...
1,information-technology-jobs,https://www.naukri.com/job-listings-urgent-opening-sr-executive-it-information-technology-ahmedabad-ecom-express-private-limited-ahmedabad-4-to-9-years-110321003742?src=jobsearchDesk&sid=16164857916548527&xp=2&px=1,Urgent Opening - Sr. Executive- IT (information Technology) -Ahmedabad,Ecom Express Private Limited,4 - 9 years,"₹ 2,50,000 - 4,50,000 P.A.",Ahmedabad,"hi, greetings from ecom express pvt ltd!!! we have an urgent opening for the post of sr. executive-it - ahmedabad job brief: ensure efficient operations of local area networks (lans) and wide area networks (wans). 1. provide user support & troubleshooting for seamless work 2. ensure efficient erp administration (user creation/deletion/modification/pod digitalization. 3. ensure proper e-mail administration (user creation/deletion/modification) 4. provide technical support (mobile app first m...","IT/Networking-Manager,","Courier, Transportation, Freight , Warehousing,",['UG :Any Graduate in Any Specialization'],"IT Software - Application Programming, Maintenance,","Full Time, Permanent",Admin/Maintenance/Security/Datawarehousing,"['LAN Troubleshooting', 'Hardware Networking', 'Networking', 'WAN', 'System Administration', 'Windows Troubleshooting', 'Information System', 'Desktop Support', 'Computer Hardware', 'Desktop Engineering', 'LAN', 'Network Administration', 'Information Technology', 'Windows System Administration']",hi greetings ecom express pvt ltd urgent opening post sr executive ahmedabad job brief ensure efficient operations local area networks lans wide area networks wans 1 provide user support troubleshooting seamless work 2 ensure efficient erp administration user creation deletion modification pod digitalization 3 ensure proper e mail administration user creation deletion modification 4 provide technical support mobile app first miles last mile deployment troubleshooting attendance machine monit...
2,information-technology-jobs,https://www.naukri.com/job-listings-information-technology-it-business-solutions-manager-procter-gamble-mumbai-8-to-10-years-160321901115?src=jobsearchDesk&sid=16164857916548527&xp=3&px

### <span style="color:#FF00FF">Extracting Tags from Job Description</span>

#### <span style="color:black">Generating Ngrams (Unigrams & Bigrams) from a corpus</span>

In [3]:
txt = job['clean_description'].to_numpy()
vectorizer = CountVectorizer(ngram_range = (1,2),min_df=0.01)
X = vectorizer.fit_transform(txt)  
features = (vectorizer.get_feature_names()) 

# Getting top ranking features 
sums = X.sum(axis = 0) 
data = [] 
for col, term in enumerate(features): 
    c = sum([1 for t in word_tokenize(term)])
    data.append( (term, c , sums[0,col])) 
ranking = pd.DataFrame(data, columns = ['term','ngram','rank']) 
words = (ranking.sort_values(['ngram','rank'], ascending = False)) 

print(words.shape)
words.sample(n=5)

(2612, 3)


,term,ngram,rank
2234,sourcing,1,23
801,electrical electronic,2,27
2462,updates,1,22
336,built,1,20
1585,noida,1,16


In [4]:
words.to_csv('../data/alltags.csv',index=False)

### <span style="color:#FF00FF">Manual Tagging</span>

#### <span style="color:black">***Manually refine the tags and assign custom NER (SKILLS/ROLES/DEPT)</span>

In [5]:
mtags = pd.read_csv('../data/manualtags.csv')
mtags.head()

,term,ngram,rank,tags
0,computer science,2,149,DEPT
1,good communication,2,142,SKILL
2,problem solving,2,133,SKILL
3,software development,2,109,SKILL
4,information technology,2,99,DEPT


### <span style="color:#FF00FF">Auto Entity Mapping</span>

In [6]:
def auto_tags(doc):
    d = dict()
    l = []
    ws = []
    st = []
    for m in mtags.to_numpy():
        w,t = m[0],m[3]
        pattern=r'\b'+w+'\\b'
        for f in re.finditer(pattern, doc):
            if w not in ws:                
                ws.append(w)
                count = len([i for i in st if (i[0] <= f.start() <= i[1]) or (i[0] <= f.end() <= i[1])]) 
                if count == 0:
                    st.append((f.start(),f.end()))
                    a = tuple([f.start(),f.end(),t.upper()])
                    l.append(a)
    d['entities'] = l
    return d

In [7]:
job['tags'] = job.apply(lambda x: auto_tags(x['clean_description']), axis=1)
job['tags'][0]

{'entities': [(323, 331, 'DEPT'),
  (30, 42, 'SKILL'),
  (684, 688, 'ROLE'),
  (535, 545, 'SKILL'),
  (43, 57, 'DEPT'),
  (104, 114, 'SKILL'),
  (282, 290, 'DEPT'),
  (766, 772, 'SKILL')]}

### <span style="color:#FF00FF">Data Preparation</span>

In [8]:
def get_train_data(df):
    tags = []
    sentences = []
    idx = []
    
    nlp=spacy.load("en_core_web_sm") 

    for j in range(len(df)):
        text = df['clean_description'][j]
        entities = df['tags'][j]
        
        # convert dictionary string to dictionary
        res = entities
        entities =  res['entities']     
        doc = nlp(text)
        tag = offsets_to_biluo_tags(doc, entities,"O")
        tmp = pd.DataFrame([list(doc), tag]).T
        loc = []
        for i in range(len(tmp)):
            if tmp[0][i].text is '.' and tmp[1][i] is 'O':
                loc.append(i)
        loc.append(len(doc))
    
        last = 0
        data = []
        for pos in loc:
            data.append([list(doc)[last:pos], tag[last:pos]])
            last = pos
    
        for d in data:
            tag = ['O' if t is '-' else t for t in d[1]]
            if len(set(tag)) > 1:
                sentences.append(d[0])
                tags.append(tag)
                idx.append(j)
    return idx,sentences, tags

In [9]:
idx,sentences, tags = get_train_data(job)
bdf = {"sentence_id":idx,"words":sentences,"labels":tags}
pdf = pd.DataFrame(bdf)
pdf = pdf.set_index(['sentence_id']).apply(pd.Series.explode).reset_index()
pdf = pdf.replace({'labels': [r'U-',r'L-']}, {'labels': 'I-'}, regex=True)

### <span style="color:#FF00FF">Export Dataset</span>

In [10]:
job.to_csv('../data/jobdescriptions_train.csv',index=False)
pdf.to_csv("../data/jobdescriptions_input.csv",index=False)